In [4]:
import time
import os
import random
import pathlib
import numpy as np                                                
import matplotlib.pyplot as plt                                   
import autograd, autograd.core, autograd.extend, autograd.tracer  
import autograd.numpy as anp      
import scipy, scipy.ndimage, scipy.sparse, scipy.sparse.linalg    

                                                     
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env import VecMonitor

%load_ext tensorboard
import tensorflow as tf
import datetime

# import the FEA Solver created by Nathan Brown: https://github.com/nkbrown503/PhDResearch/tree/main
from FEA_Solver import *




In [ ]:
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d


# Manage the problem setup parameters   
def get_args(normals, forces, density=1e-4):
    width = normals.shape[0] - 1
    height = normals.shape[1] - 1
    
    fixdofs = np.flatnonzero(normals.ravel())
    alldofs = np.arange(2 * (width + 1) * (height + 1))
    freedofs = np.sort(list(set(alldofs) - set(fixdofs)))
   
    params = {
      # material properties
      'young': 1, 'young_min': 1e-9, 'poisson': 0.3, 'g': 0,
      # constraints
      'density': density, 'xmin': 0.001, 'xmax': 1.0,
      # input parameters
      'nelx': width, 'nely': height, 'mask': 1, 'penal': 3.0, 'filter_width': 1,
      'freedofs': freedofs, 'fixdofs': fixdofs, 'forces': forces.ravel(),
      # optimization parameters
      'opt_steps': 80, 'print_every': 10}
    return ObjectView(params)


# textbook beam example
def mbb_beam(width=6, height=6, density=1e-4, y=1, x=0, rd=-1):  
    normals = np.zeros((width + 1, height + 1, 2))
    normals[0, 0, x] = 1
    normals[0, 0, y] = 1
    normals[0, -1, x] = 1
    normals[0, -1, y] = 1
    forces = np.zeros((width + 1, height + 1, 2))
    forces[-1, rd, y] = 1
    return normals, forces, density

In [ ]:

width = 10
length = 10
train_steps = 1000
log_dir =  pathlib.Path("logs/")


# Formalising the Problem
Beim Durchführen einer Topologieoptimierung geht es darum folgende Funktion zu mimimieren

$$\begin{aligned}\mathbf{\mathit{F}} \big( \mathbf{u}(\rho), \rho \big) = \int_\Omega^\Omega{f(\mathbf{u}(\rho), \rho)}\,\mathrm{d}V\end{aligned}$$


mit Einbezug der Funktionen
$$\begin{aligned}G_{0}(\rho)=\int_\Omega^\Omega\rho\mathrm{d}V-V_{0}\end{aligned}\leq 0$$
und
$$\begin{aligned}\rho(x) \in \{0,1\}\,   \end{aligned} \forall x \in \Omega$$





Der zu trainierende Agent benötigt einen Zustandsraum, mithilfe dessen der zu trainierende Agent alle möglichen Beobachtungen innerhalb der Trainingsumgebung sehen kann.\
Zur Repräsentation des Zustandsraums des Grids sollte eine NxNx3 Matrix verwendet werden
$$\begin{aligned}O_{d,x, y}\end{aligned}$$
Die ersten beiden Indizes $x,y$ repräsentieren so die Koordinaten der einzelnen Punkte, 
und der letzte index funktioniert folgendermaßen:
- für d = 0 zeigt die Matrix die Spannung jedes Elements. Berechnet wird diese durch das Inverse der normalisierten Von Mises Formel:
    $$\sigma_{\mathit{VM}} = \sqrt{\sigma_{x}^{2} + \sigma_{y}^{2} - \sigma_{x}\sigma_{y} + 3\tau_{x,y}}$$ 
    Also:
    $$O_{0,x,y} = \left(\frac{\sigma_{\mathit{VM,x,y}}}{\sigma_{\mathit{VM,max}}}\right)^{-1}$$
- für d = 1 enspricht $O_{1,x,y}$ einer Boolschen repräsentation, ob der Knoten an Position $(x,y)$ nach den Dirichlet-Randbedingungen fixiert ist oder nicht
    Also:
    $$O_{1,x,y} \in \{1,0\}\,   \forall (x,y)$$
- für d = 1 enspricht $O_{3,x,y}$ analog einer boolschen repräsentation, ob der Knoten eine Kraft erfährt
    Also auch:
    $$O_{3,x,y} \in \{1,0\}\,   \forall (x,y)$$





# Objective Function
$\mathbf{\mathit{F}} \big( \mathbf{u}(\rho), \rho \big) = \int_\Omega^\Omega{f\big(\mathbf{u}(\rho), \rho\big)}\,\mathrm{d}V$

In [ ]:
def objective(rho, f, u, volume):
    # Berechne u als Funktion von rho, hier vereinfacht dargestellt
    u_rho = u(rho)
    return np.sum(f(u_rho, rho) * volume)

# Compliance Function


In [ ]:
def volume_constraint(rho, volume, V0):
    return V0 - np.sum(rho * volume)

In [ ]:
def calculate_von_mises_stress(stiffness_matrix, strain):
    
    stresses = anp.dot(stiffness_matrix, strain)
    
    sigma_x = stresses[0]
    sigma_y = stresses[1]
    tau_xy = stresses[2]

    von_mises_stress = anp.sqrt(sigma_x**2 
                                - sigma_x*sigma_y 
                                + sigma_y**2 
                                + 3*tau_xy**2)

    return von_mises_stress

$$\sigma_{\mathit{VM}} = \sqrt{\sigma_{x}^{2} + \sigma_{y}^{2} - \sigma_{x}\sigma_{y} + 3\tau_{x,y}}$$ 

In [ ]:
def calculate_stress(stiffness_matrix, strain, obs):
    stress_matrix= obs[:,:,0]
    for x in np.nditer(stress_matrix, op_flags=['readwrite']):
        if x > 0:
            x[...] *= calculate_von_mises_stress()
    return stress_matrix

In [ ]:
Z = [[1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 1, 1, 0],
     [1, 0, 0, 1]]

fig, ax = plt.subplots()
ax.imshow(Z, cmap='GnBu')

plt.show()

In [5]:
class OptimisingEnv(gym.Env):
    
    metadata = {"render.modes" : ["human"]}
    

    def __init__(self, render_mode=None, width=10, length=10):
        
        # getting the Parameters from our Beam Example
        self.args = get_args(*mbb_beam(0))
        
        # setting a few defualt parameters for the solver
        self.Lx = 1
        self.Ly = 1
        self.p = 10

        # adjusting the way Normals and Forces are Represented for the FEA Solver
        matrix1 = self.args.normals[:, :, 0]
        matrix2 = self.args.normals[:, :, 1]
        self.BC_nodes = np.logical_or(matrix1, matrix2).astype(int)
        matrix1 = self.args.forces[:, :, 0]
        matrix2 = self.args.forces[:, :, 1]
        self.LC_Nodes = np.logical_or(matrix1, matrix2).astype(int)

        # creating bith the observation and action space
        self.observation_space = spaces.Box(low=0, high=1, shape=(width, length, 3))
        self.action_space = spaces.Discrete(width*length)
        
        self.reward = 0
        self.Counter = 0
        self.needs_reset = True


    def step(self, action):
        self.Counter+=1 

        rs_place=self.VoidCheck[int(action)]
        self.VoidCheck[int(action)]=0
        
        ElementMat=np.reshape(self.VoidCheck,(self.args.nelx,self.args.nely))
        # Check if the ElementMat is made up of a single group
        SingleCheck=isolate_largest_group_original(ElementMat)
        
        if self.needs_reset:
            raise RuntimeError("Tried to step environment that needs reset")
        
        if rs_place==1 and action not in self.BC and SingleCheck[1]==True:
            done = False
        self.observation_space = self._get_obs()
        
        
        
        if done:
            self.needs_reset = True
        truncated = False
        return self.observation_space, self.reward, done, truncated, {}



    def reset(self):
        #get the current Observation Space
        self.observation_space = self._get_obs()
        # Gymnasium always wants you to return info
        info = {"info" : "nothing special"}
        # resetting variables
        self.reward=0
        self.Counter=0
        self.needs_reset = False
        return self.observation_space
    

    def _get_obs(self):
        # Runs the FEA Solver and Saves the results to the Observation Space
        self.Results = FEASolve(self.args, list(self.VoidCheck),self.Lx,self.Ly,
                             self.LC_Nodes,self.Load_Directions,
                             self.BC_Nodes,Stress=True)
        #Saving the Von Mises Stress Array
        self.Stress_state = self.Results[3]
        self.P_Norm = sum(sum([number**self.p for number in np.reshape
                             (self.Results[2],(1,self.nelx*self.nely))]))**(1/self.p)        
        self.Stress_state = np.reshape(self.Stress_state,
                                     (self.args.nelx,self.args.nely))  
        # Updating the Observation Space Matrix      
        self.observation_space[:,:,0] = self.Stress_state
        self.observation_space[:,:,1] = self.BC_state
        self.observation_space[:,:,2] = self.LC_state
        return self.observation_space

In [2]:
   
def combine_matrices(matrix):

    matrix1 = matrix[:, :, 0]
    matrix2 = matrix[:, :, 1]

    result = np.logical_or(matrix1, matrix2).astype(int)

    return result

IndentationError: unindent does not match any outer indentation level (<string>, line 8)